<a href="https://colab.research.google.com/github/ray-hsiehjy/SF_restaurant-inspections/blob/master/SF_restaurant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import folium
from folium import plugins

%matplotlib inline

In [0]:
df = pd.read_csv("/content/restaurant-scores-lives-standard.csv")
df.head()

,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_location,business_phone_number,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category
0,101192,Cochinita #2,2 Marina Blvd Fort Mason,San Francisco,CA,NaN,NaN,NaN,NaN,1.415043e+10,101192_20190606,2019-06-06T00:00:00.000,NaN,New Ownership,NaN,NaN,NaN
1,97975,BREADBELLY,1408 Clement St,San Francisco,CA,94118,NaN,NaN,NaN,1.415724e+10,97975_20190725,2019-07-25T00:00:00.000,96.0,Routine - Unscheduled,97975_20190725_103124,Inadequately cleaned or sanitized food contact...,Moderate Risk
2,92982,Great Gold Restaurant,3161 24th St.,San Francisco,CA,94110,NaN,NaN,NaN,NaN,92982_20170912,2017-09-12T00:00:00.000,NaN,New Ownership,NaN,NaN,NaN
3,101389,HOMAGE,214 CALIFORNIA ST,San Francisco,CA,94111,NaN,NaN,NaN,1.415488e+10,101389_20190625,2019-06-25T00:00:00.000,NaN,New Construction,NaN,NaN,NaN
4,85986,Pronto Pizza,798 Eddy St,San Francisco,CA,94109,NaN,NaN,NaN,NaN,85986_20161011,2016-10-11T00:00:00.000,NaN,New Ownership,85986_20161011_103114,High risk vermin infestation,High Risk


In [0]:
# convert inspection date to datetime
df["inspection_date"] = pd.to_datetime(df["inspection_date"], yearfirst=True)

# violation_id is business_id + date + citation code (alias violation description)
# split column and keep only the citation code
df["violation_id"] = df["violation_id"].str.split("_", expand=True).iloc[:,-1]

# convert to ordinal 
# if df["risk_category"].dtype == "object":
#     df["risk_category"].replace({np.nan:0, 'Moderate Risk':2, 'High Risk':3, 'Low Risk':1}, inplace=True)

df.head()

,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_location,business_phone_number,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category
0,101192,Cochinita #2,2 Marina Blvd Fort Mason,San Francisco,CA,NaN,NaN,NaN,NaN,1.415043e+10,101192_20190606,2019-06-06,NaN,New Ownership,NaN,NaN,NaN
1,97975,BREADBELLY,1408 Clement St,San Francisco,CA,94118,NaN,NaN,NaN,1.415724e+10,97975_20190725,2019-07-25,96.0,Routine - Unscheduled,103124,Inadequately cleaned or sanitized food contact...,Moderate Risk
2,92982,Great Gold Restaurant,3161 24th St.,San Francisco,CA,94110,NaN,NaN,NaN,NaN,92982_20170912,2017-09-12,NaN,New Ownership,NaN,NaN,NaN
3,101389,HOMAGE,214 CALIFORNIA ST,San Francisco,CA,94111,NaN,NaN,NaN,1.415488e+10,101389_20190625,2019-06-25,NaN,New Construction,NaN,NaN,NaN
4,85986,Pronto Pizza,798 Eddy St,San Francisco,CA,94109,NaN,NaN,NaN,NaN,85986_20161011,2016-10-11,NaN,New Ownership,103114,High risk vermin infestation,High Risk


In [0]:
m = folium.Map(location=[37.7749, -122.4194], zoom_start=13)

gps = df[["business_id", "business_latitude", "business_longitude"]].dropna().drop_duplicates().set_index("business_id")

for r in ["High Risk", "Moderate Risk", "Low Risk"]:
    risk_df = df[(df.business_latitude.notna())].groupby("business_id")["risk_category"].value_counts()
    risk_df = pd.DataFrame(risk_df.loc[(slice(None), r)])
    risk_df = risk_df.join(gps, on="business_id", how="left")
    risk_array = risk_df[["business_latitude", "business_longitude", "risk_category"]].to_numpy()

# plot heatmap
    plugins.HeatMap(risk_array, radius=10, name=r).add_to(m)  #max_val=1, 

folium.LayerControl().add_to(m)

m